census resources:
- example blockgroup: https://api.census.gov/data/2019/pdb/blockgroup/examples.html
- api document: https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf
- handbook: https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_api_handbook_2020_ch02.pdf
- variables: https://api.census.gov/data/2019/acs/acs5/variables.html
- data surveys: https://api.census.gov/data.html
- query exmaples: https://www.census.gov/data/developers/guidance/api-user-guide/query-examples.html
- geoindentifiers: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html
- tutorial: https://github.com/datamade/census
- api intro: https://www.census.gov/data/developers/data-sets/acs-5year.html

In [3]:
import tarfile
import pandas as pd
import numpy as np
import os
import requests
import dotenv

In [4]:
# load key for API
dotenv.load_dotenv()
key = os.getenv('census_key')

In [5]:
# load ems data and get coordinates
emsdata = pd.read_csv('locationData.csv')
emsdata['Coordinates']=list(zip(emsdata['IncidentLatitude'],emsdata['IncidentLongitude']))

In [10]:
# API for getting blocks from geocode
# Specify call and parameters
search_url = 'https://geo.fcc.gov/api/census/area?'

# get coordinates for lookup
coord = emsdata[~((emsdata['IncidentLatitude'].isna())|emsdata['IncidentLongitude'].isna())]['Coordinates'].drop_duplicates()

# fetch data and save as json
blockmap = [requests.get(search_url,params={'lat': la, 'lon': lo}) \
     .json()['results'][0]['block_fips'] \
     for la, lo in coord]

In [12]:
# store block level data in dataframe
blocks = pd.DataFrame({
    'Coordinates':coord,
    'BlockGroup':blockmap
})

In [14]:
# merge block data to EMS
emsdata_bg = emsdata.merge(blocks, left_on='Coordinates', right_on='Coordinates')
emsdata_bg['BlockGroup12'] = emsdata_bg['BlockGroup'].str[:12]
emsdata_bg.to_csv('emsmerged.csv')

In [6]:
# export ems data merged with block data
emsdata_merged = pd.read_csv('emsmerged.csv')

In [15]:
# get unique blocks
emsdata_merged['BlockGroup12'] = emsdata_merged['BlockGroup'].astype(str).str[:12]
blockUnique = emsdata_merged['BlockGroup12'].drop_duplicates()

In [26]:
#Block info
st = blockUnique[0][0:2]
co = blockUnique[0][2:5]
tr = blockUnique[0][5:11]
bl = blockUnique[0][11:]

In [29]:
# API for getting census-level data
from census import Census
#from us import states

c = Census(key)

#block group level income data
blockIncome = [c.acs5.get(('NAME', 'B19013_001E'),
          {'for': 'block group:{}'.format(b[11:]),
          'in':'state:{} county:{} tract:{}'.format(b[0:2], b[2:5], b[5:11])})[0]['B19013_001E']
for b in blockUnique]

In [30]:
censusBlock = pd.DataFrame({
    'Block': blockUnique,
    'Income': blockIncome
})

censusBlock

,Block,Income
0,515400005023,98967.0
1,515400004012,61932.0
2,515400002022,12000.0
3,515400008004,51250.0
4,515400004023,36670.0
...,...,...
20698,510030111003,-666666666.0
21353,510030102012,88672.0
21369,510030105001,-666666666.0
21868,510030111004,99333.0
